In [20]:
from jetbot import Camera
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from IPython.display import display
import cv2

In [21]:
from servoserial import ServoSerial

servo_device = ServoSerial()
servo_device.Servo_serial_double_control(1, 2400, 2, 2000)

serial Open!
serial Close!


In [22]:
!wget -q -N https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

In [23]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

In [25]:
def face_detect(img):
    face_ptr = None
    # 그레이스케일로 변경 후 검출, 바운딩 박스는 원본 이미지(컬러)에 그려주면 됨
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    if len(faces) > 0:
        (face_x, face_y, face_w, face_h) = faces[0]
        # 사각형 박스 표시
        cv2.rectangle(img, (face_x, face_y), (face_x+face_w, face_y+face_h), (0,255,0), 4)
        # 중심좌표 계산
        face_cx = face_x + (int(face_w/2))
        face_cy = face_y + (int(face_h/2))
        face_ptr = (face_x, face_y, face_w, face_h, face_cx, face_cy)
        cv2.circle(img, (face_cx, face_cy), 5, (0,0,255))

    return img, face_ptr

In [26]:
camera = Camera.instance(width=720, height=720)

In [27]:
widget = widgets.Image(format='jpeg', width=300, height=300)
display(widget)

Image(value=b'', format='jpeg', height='300', width='300')

In [11]:
import PID
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

In [30]:
while 1:
    try:
        frame = camera.value
        frame = cv2.resize(frame, (300, 300))
        image, face_ptr = face_detect(frame)
        if face_ptr != None:
            xservo_pid.SystemOutput = face_ptr[4]
            xservo_pid.SetStepSignal(150)
            xservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuex = int(2048+xservo_pid.SystemOutput)
            
            yservo_pid.SystemOutput = face_ptr[5]
            yservo_pid.SetStepSignal(150)
            yservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuey = int(2048+yservo_pid.SystemOutput)
            
            servo_device.Servo_serial_double_control(1, target_valuex, 2, target_valuey)
            
        widget.value = bgr8_to_jpeg(frame)
               
       
    except KeyboardInterrupt:
        break

camera.unobserve_all()